In [1]:
from google.colab import files
uploaded = files.upload()


Saving Simpsons.zip to Simpsons.zip


In [2]:
import os
print(os.listdir('/content'))  # List files in the current directory to confirm the file is uploaded


['.config', 'Simpsons.zip', 'sample_data']


In [3]:
import zipfile

# Path to the uploaded ZIP file
zip_path = '/content/Simpsons.zip'  # Path to your uploaded file

# Extract the ZIP file to a specific directory (e.g., /content/Simpsons/)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/Simpsons')  # Extract the files to /content/Simpsons

# Verify the extracted files
import os
print(os.listdir('/content/Simpsons'))


['__MACOSX', 'archive']


In [10]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import os

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to check if a file is a valid image and not a macOS system file
def is_image_file(file_path):
    file_name = os.path.basename(file_path)
    return file_name.lower().endswith(('.jpg', '.jpeg', '.png')) and not file_name.startswith('._')

# Data Preprocessing with Augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation transform without augmentation
val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Dataset
data_path = './Simpsons'  # Path to your dataset
full_data = datasets.ImageFolder(
    root=os.path.join(data_path, 'archive/characters_train'),
    transform=train_transform
)

# Filter out macOS system files
full_data.samples = [
    sample for sample in full_data.samples if is_image_file(sample[0])
]

# Split into train and validation sets (80/20 split)
train_size = int(0.8 * len(full_data))
val_size = len(full_data) - train_size
train_data, val_data = random_split(full_data, [train_size, val_size])

# Update validation dataset to use val_transform
val_data.dataset.transform = val_transform

print(f"Total samples: {len(full_data)}")
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Number of classes: {len(full_data.classes)}")

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# Define Improved CNN Model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Block 1
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        # Block 2
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        # Block 3
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        # Block 4
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))

        # Flatten
        x = x.view(x.size(0), -1)

        # FC layers with dropout
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize Model, Loss, and Optimizer
model = CNN(num_classes=len(full_data.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Learning Rate Scheduler: Reduce learning rate on plateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Validation function
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Train the Model
epochs = 20
best_val_acc = 0.0

# Directory for saving model
save_dir = './Simpsons/models'
os.makedirs(save_dir, exist_ok=True)

print("\nStarting Training...")
print("-" * 60)

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    # Step the scheduler based on validation loss
    scheduler.step(val_loss)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print("-" * 60)

    # Save best model based on validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, 'model_best.pth'))
        print(f"  ✓ New best model saved! Val Acc: {val_acc:.2f}%")

# Save the final model
torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))

# Save the class names to a JSON file
with open(os.path.join(save_dir, 'classes.json'), 'w') as f:
    json.dump(full_data.classes, f)

print("\nTraining Complete!")
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"Model saved as 'model.pth' and 'model_best.pth' in '{save_dir}'")
print("Classes saved as 'classes.json' in the same directory.")


Using device: cuda
Total samples: 16764
Training samples: 13411
Validation samples: 3353
Number of classes: 42

Starting Training...
------------------------------------------------------------
Epoch [1/20]
  Train Loss: 3.3493 | Train Acc: 10.72%
  Val Loss: 2.9940 | Val Acc: 14.73%
------------------------------------------------------------
  ✓ New best model saved! Val Acc: 14.73%
Epoch [2/20]
  Train Loss: 2.9671 | Train Acc: 15.76%
  Val Loss: 2.7152 | Val Acc: 21.29%
------------------------------------------------------------
  ✓ New best model saved! Val Acc: 21.29%
Epoch [3/20]
  Train Loss: 2.8223 | Train Acc: 18.50%
  Val Loss: 2.6073 | Val Acc: 24.16%
------------------------------------------------------------
  ✓ New best model saved! Val Acc: 24.16%
Epoch [4/20]
  Train Loss: 2.7067 | Train Acc: 21.33%
  Val Loss: 2.4598 | Val Acc: 28.51%
------------------------------------------------------------
  ✓ New best model saved! Val Acc: 28.51%
Epoch [5/20]
  Train Loss: 2.5